<a href="https://colab.research.google.com/github/mrarat76/Tensorflow_Practises/blob/main/NLP_Practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## recurrent neural network

## LSTM rnn hücrelerine denir "tanh" non-linearity eklemek için kullnaırız


## Outputta sigmodi kullanıyoruz


NLP'nin amacı bilgiyi doğal dilden elde etmektir( metini veya konuşmayı sıralamak gibi)

NLP genel manada seq2seq problemlerine odaklanır.


**Seq2seq**, kısaltması "sequence to sequence" (diziden diziye) anlamına gelir ve genellikle doğal dil işleme (NLP) ve makine öğrenmesinde kullanılan bir model türüdür. Bu model, bir girdi dizisini (örneğin bir cümle veya kelime dizisi) alır ve başka bir diziye (genellikle başka bir dilde bir cümle veya metin) çevirir. Seq2seq modelleri, özellikle makine çevirisi, otomatik özetleme, soru-cevap sistemleri ve sohbet robotları gibi alanlarda yaygın olarak kullanılır.

Seq2seq modeli tipik olarak iki ana bileşenden oluşur:

Encoder (Kodlayıcı): Girdi dizisini alır ve bu diziyi sabit boyutlu bir vektör temsiline dönüştürür. Bu vektör, girdi dizisinin bir tür özeti olarak düşünülebilir ve genellikle diziye ait önemli bilgileri içerir.

Decoder (Çözücü): Encoder tarafından oluşturulan sabit boyutlu vektörü alır ve hedef diziye dönüştürür. Çözücü, bu vektörü kullanarak adım adım çıktı dizisini üretir.

Bu modeller genellikle tekrarlayan sinir ağları (RNN), uzun kısa süreli bellek ağları (LSTM) veya gated recurrent units (GRU) gibi yapıları kullanır. Son yıllarda, daha gelişmiş seq2seq modelleri, dikkat mekanizmaları (attention mechanisms) ve Transformer mimarileri gibi teknikleri kullanarak daha iyi performans sergilemiştir. Dikkat mekanizması, modelin girdi dizisinin hangi bölümlerine odaklanması gerektiğini daha iyi anlamasını sağlar ve böylece daha doğru çeviriler yapılmasına olanak tanır.







In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d215a27f-13c3-8ac0-1ee1-9862ab91333a)


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2024-03-10 07:56:45--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-03-10 07:56:46 (88.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-03-10 07:56:50--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.207, 172.253.118.207, 74.125.200.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   740KB/s    in 0.8s    

2024-03-10 07:56:51 (740 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
### Visualizing a text dataset

import pandas as pd

train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

train_df.head()



,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
## Dataframi shuffellama

train_df_shuffled =train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()



,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
## How many total samples?


len(train_df), len(test_df)

(7613, 3263)

In [12]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target =row
  print(f"Target:{target}", "(real_diaster)" if target > 0 else "(not real diaster)")
  print(f"Text\n{text}\n")
  print("---\n")



Target:0 (not real diaster)
Text
@Grazed @invalid @Stretcher @Rexyy @Towel 'Ben favorited'

---

Target:1 (real_diaster)
Text
Two giant cranes holding a bridge collapse into nearby homes http://t.co/9asc1hhFNJ

---

Target:1 (real_diaster)
Text
Riot police intervene after Southampton and Vitesse Arnhem supporters clash: Û¢ Fans clash in buildup to second... http://t.co/sKVNmtZGeG

---

Target:1 (real_diaster)
Text
Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/3sNyOOhseq

---

Target:0 (not real diaster)
Text
Downtown Emergency Service Center is hiring! #Chemical #Dependency Counselor or Intern in #Seattle apply now! #jobs http://t.co/SKQPWSNOin

---



In [13]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [14]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [17]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [18]:
max_vocab_lenght= 10000
max_length=15

text_vectorizer= TextVectorization(max_tokens= max_vocab_lenght,
                                   output_mode="int",
                                   output_sequence_length= max_length)


In [19]:
## Fit the text vectorizer

text_vectorizer.adapt(train_sentences)


sample_sentence="There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Random sentence choose


random_sentence = random.choice(train_sentences)
print(f"Original text: \n {random_sentence}\
\n\nVectorized version:")

text_vectorizer([random_sentence])

Original text: 
 @PTSD_Chat Yes. I feel the root of that is Shame - which can be found in the rubble of most trauma. #PTSDchat

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[3505,  635,    8,  214,    2, 1998,    6,   16,    9, 1595,  234,
          71,   21,  225,    4]])>

In [21]:
## Get the unique words in the vocab


words_in_vocab =text_vectorizer.get_vocabulary()

top_5_words = words_in_vocab[5:]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'youtube', 'dead', 'two', 'families', '3', 'train', 'full', 'being', 'war', 'many', 'today', 'think', 'only',

In [22]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

Yukarıda yaptıklarımız tokenization layer oluyor.


## Embedding ve Tokenization FARKI


Embedding ve tokenization, doğal dil işleme (NLP) alanında kullanılan iki farklı teknikdir. İkisi de metin verilerini makine öğrenimi modellerine veya yapay zeka sistemlerine giriş olarak kullanılabilir. Ancak işlevleri ve kullanımları farklıdır.

**Tokenization**:
Tokenization, metin verilerini belirli bir format veya dil bilgisine göre parçalara ayırma işlemidir. Bu parçalar "token" olarak adlandırılır. Tokenization sırasında, metin cümleler veya kelimeler gibi daha küçük parçalara bölünür. Örneğin, bir cümle cümlelerine veya kelimelerine ayrılabilir.
Tokenization, metnin yapısını anlamak, kelime sıklığı sayımı yapmak, dil modelleri oluşturmak veya bir metin veri kümesini işlemek için kullanılır. Örnek olarak, bir cümleyi tokenize ederken, "Bu bir örnek cümledir." gibi bir giriş "Bu", "bir", "örnek", "cümledir" gibi ayrı ayrı parçalara bölünebilir.

**Embedding**:
Embedding, metin veya kategorik verileri makine öğrenimi modellerine veya yapay zeka sistemlerine giriş olarak temsil etmek için kullanılan bir tekniktir. Embedding, tokenization'ın ötesine geçerek, kelime veya kategori gibi girişlerin bir vektör uzayında temsil edilmesini sağlar. Bu vektör temsilleri, kelimenin veya kategorinin anlamsal benzerliklerini veya ilişkilerini yansıtacak şekilde öğrenilir.
Özellikle kelime gömme (word embedding) sıkça kullanılan bir embedding türüdür. Burada, her kelime bir vektörle temsil edilir ve bu vektörler, genellikle kelimenin anlamını yansıtacak şekilde eğitilir. Örneğin, "kral" ve "kraliçe" kelimeleri arasındaki ilişkiyi temsil eden vektörler birbirlerine daha yakın olacaktır.

Sonuç olarak, tokenization, metin verilerini işlemek ve parçalara ayırmak için kullanılırken, embedding, metin verilerini sayısal temsillerle temsil etmek ve anlamsal ilişkileri yansıtmak için kullanılır. Birçok durumda, öncelikle metin verileri tokenize edilir ve ardından bu tokenler embedding teknikleri kullanılarak sayısal temsillerle değiştirilir. Bu, makine öğrenimi modellerinin metin verilerini anlaması ve işlemesi için önemli bir adımdır.

In [23]:
### Embedding layer
tf.random.set_seed(42)

max_vocab_length = 10000  # Example value, replace with the actual size of your vocabulary

from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding


In [24]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
The eyes of the nation &amp; broader conservation community are on #Alaska @AKGovBillWalker reinstate emergency buffer #ProtectDenaliWolves      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04615417,  0.00094073, -0.01951014, ..., -0.03779303,
         -0.03436442, -0.00923699],
        [-0.02000662,  0.03612311, -0.03440069, ..., -0.01869966,
         -0.00996081, -0.01857265],
        [ 0.01368413, -0.04759178,  0.00064143, ...,  0.04443247,
         -0.01912336,  0.01629626],
        ...,
        [-0.04470298,  0.03987129,  0.03077816, ..., -0.01368573,
         -0.04999148,  0.03866   ],
        [-0.04477834, -0.00936449, -0.02273398, ..., -0.03096161,
         -0.04285662,  0.01482994],
        [ 0.00371026, -0.03762941,  0.02307493, ...,  0.04499269,
          0.01665919, -0.03293301]]], dtype=float32)>

In [25]:
sample_embed[0][0]


<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 4.61541750e-02,  9.40728933e-04, -1.95101388e-02,  1.57535076e-04,
       -3.55335250e-02,  3.48954238e-02, -2.50693094e-02,  2.47529633e-02,
       -2.58360747e-02,  2.96977274e-02,  2.89735235e-02, -4.86048311e-03,
       -1.67359933e-02, -3.41512784e-02, -4.88299839e-02, -1.36673227e-02,
        3.41912545e-02, -1.99980624e-02,  1.95250660e-03, -2.65193339e-02,
        3.11781801e-02,  4.11108769e-02, -5.45009971e-05,  2.16660611e-02,
       -3.96572948e-02, -2.43419055e-02, -3.76408212e-02,  3.96177433e-02,
        1.22664124e-03, -2.70609260e-02, -4.34588194e-02, -3.47643495e-02,
       -4.67101820e-02, -3.10615785e-02,  1.11086257e-02,  4.82572578e-02,
       -3.39705236e-02, -4.09869552e-02, -2.93011554e-02, -1.26821622e-02,
       -4.83255275e-02, -4.38016318e-02,  1.68272890e-02, -9.73308086e-03,
        1.16081722e-02, -1.38460509e-02,  2.95232795e-02, -2.19154712e-02,
       -4.30760868e-02, -2.92376280e-02, -5.87915257

In [26]:
## Text verisini modelleme (running a series of experiments)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [29]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [30]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1

In [31]:
from helper_functions import create_tensorboard_callback

SAVE_DIR="model_logs"

In [32]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens) Her bir token için tahminleme yapar eğer global average pool yapmaz isek.
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [33]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [35]:
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20240310-075658
Epoch 1/5
215/215 [==============================] - 15s 49ms/step - loss: 0.6096 - accuracy: 0.6938 - val_loss: 0.5364 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4420 - accuracy: 0.8181 - val_loss: 0.4691 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3471 - accuracy: 0.8603 - val_loss: 0.4588 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2855 - accuracy: 0.8911 - val_loss: 0.4639 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2386 - accuracy: 0.9123 - val_loss: 0.4764 - val_accuracy: 0.7861


In [36]:
model_1.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 4ms/step - loss: 0.4764 - accuracy: 0.7861


[0.47635796666145325, 0.7860892415046692]

In [37]:
embedding.weights


[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.02603087, -0.00526017,  0.04216257, ...,  0.02106043,
         -0.04842012,  0.04568238],
        [-0.04627642,  0.04146292,  0.03074089, ..., -0.01362448,
         -0.04950592,  0.03917032],
        [ 0.0264951 ,  0.02040879,  0.00183451, ..., -0.05885037,
         -0.05533161,  0.01087261],
        ...,
        [ 0.04774226,  0.02956494, -0.01909548, ..., -0.01341869,
          0.02109681,  0.00863089],
        [-0.06719888,  0.04442936,  0.08669272, ..., -0.03156506,
          0.00542753,  0.08226341],
        [-0.08274867,  0.04555938,  0.04133812, ..., -0.04190687,
         -0.09203064,  0.08547501]], dtype=float32)>]

In [40]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)


(10000, 128)


In [41]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 6ms/step


array([[0.41547754],
       [0.74377704],
       [0.9977022 ],
       [0.11305733],
       [0.10647073],
       [0.93252957],
       [0.91021955],
       [0.9929804 ],
       [0.96798164],
       [0.26155126]], dtype=float32)

In [42]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [43]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [44]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [45]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.61, Difference: -0.66
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


In [46]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 4ms/step


array([[0.41547754],
       [0.74377704],
       [0.9977022 ],
       [0.11305733],
       [0.10647073],
       [0.93252957],
       [0.91021955],
       [0.9929804 ],
       [0.96798164],
       [0.26155126]], dtype=float32)

In [47]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [48]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results


{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [49]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [50]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.61, Difference: -0.66
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


In [51]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [52]:
model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [53]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)
